In [5]:
from sklearn import datasets, svm, metrics 
digits = datasets.fetch_openml('mnist_784')

In [6]:
import matplotlib.pyplot as plt
import heapq
from numpy.linalg import norm 


from sklearn.neighbors import KNeighborsClassifier
from sklearn_algor import split, find_f_1, euclid_dist, classify

## My Implementation

In [7]:
#list of tuples, where first tuple is image and second tuple is classifaction
images_label_list = list(zip(digits.data, digits.target))
split_dict = split(images_label_list)
n_samples = len(digits.data)
data = digits.data.reshape((n_samples, -1))


#work on a small sample
k = 500
j = 1000

predictions = knn(images_label_list[:500], list(digits.data[k:j]), 3)
my_f1_dict = find_f_1(predictions, digits.target[k:j])

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
for number in list(my_f1_dict.values()):
    print('{:.2f} & '.format(number), end = '')

## SciKitLearn

In [ ]:
#define Model
model = KNeighborsClassifier(n_neighbors=3)
model.fit(split(data)['train'], split(digits.target)['train'])
#model.fit(data[:int(n_samples * .8)], digits.target[:int(n_samples * .8)])
y_hat = model.predict(data[int(n_samples * .8):int(n_samples * .95)])

In [ ]:
y = digits.target[int(n_samples * .8):]
f_1_dict = find_f_1(y_hat,y)

In [ ]:
print(f_1_dict)
print('min is ', min(list(f_1_dict.values())))
print('max is ', max(list(f_1_dict.values())))
# print(type(images_label_list[3][0]))
# print(type(digits.data))

In [ ]:
for number in list(f_1_dict.values()):
    print('{:.2f} & '.format(number), end = '')

## Modules


In [ ]:
def knn(train, pred, k):
    """
    Uses the train data to make predictions on the test data
    :param train: zipped list of tupules, 1st element picture array, second element classification eg "1"
    :param pred: just the picture array, we want to predict the classification for this array/picture
    :param k: number of near neighbors to consider
    """
    inf = 10 ** 6  # ~inf
    prediction_list = []
    
    for i in range(len(pred)):
        nn_heap = []
        #initialize a new heap for each data point
        for _ in range(k):# use negatives to turn the implemented min heap into a max heap
            heapq.heappush(nn_heap, (-1 * inf, '1'))  # initialize the set of NN to five 1's at infinite distance
        
        for j in range(len(train)):
            dist = euclid_dist(pred[i], train[j][0])
            if dist < -1 * nn_heap[0][0]:
                heapq.heappushpop(nn_heap, (-1 * dist, train[j][1]))  # add distance and classification
            assert len(nn_heap) == k
        prediction_list.append(classify(nn_heap))
    return prediction_list

In [ ]:
def classify(nn_tup_list):
    """

    :param nn_tup_list: a list of tuple of neighbors and their distance from the data point
    :return:
    """
    count_list = [0] * 10
    neighbors = list(zip(*nn_tup_list))[1]
    distances = list(zip(*nn_tup_list))[0]
    for i in range(10):  # count the instances of each prediction
        for neighbor in neighbors:
            if neighbor == str(i):
                count_list[i] += 1
    maximum = max(count_list)  # most common prediction, (min to account for negative values in the max heap)
    if maximum == 1:
        return neighbors[distances.index(max(distances))]
    return str(count_list.index(maximum))